In [1]:
%load_ext autoreload 
%autoreload 2

from imports import *
import queue_system as qs
from queue_system import *

seed = random.seed(10)
seed_np = np.random.seed(10)

# Aggregation

In [2]:
with open('Q/Q_start.pkl', "rb") as fp:
    Q_start = pickle.load(fp)

In [3]:
weights_dict = {}
model_agg = Aggregation(graph, arrival_rates, experts, eta, queue_max)
model = StochasticMatching(graph, arrival_rates, queue_max)

len_updates = 500
res = int(len_updates / 100)
num_repeat_est = 1

weights_dict['-1'] = {state: np.array([1/K] * K) for state in state_space_ind.keys()}

repeat_weights = 10
learning_rate = 0.00014

In [4]:
%%time 
random.seed(6)
np.random.seed(6)
for n in range(repeat_weights):
    weights_dict = {}
    model_agg = Aggregation(graph, arrival_rates, experts, eta, queue_max, Q_start)
    model = StochasticMatching(graph, arrival_rates, queue_max)
    weights_dict['-1'] =  np.ones([num_states, K]) / K 
    
    for i in tqdm(range(len_updates + 1)):
        if (i <= 200) and (i%10 == 0):
            weights = model_agg.aggregation_update_exp(discount, learning_rate, model, H, k, num_repeat_est)
            weights_dict[str(i)] = weights.copy()
            random_state = state_space[random.randint(0, len(state_space)-1)]
            
        elif (i%res == 0) and (i>200):
            weights = model_agg.aggregation_update_exp(discount, learning_rate, model, H, k, num_repeat_est)
            weights_dict[str(i)] = weights.copy()
            random_state = state_space[random.randint(0, len(state_space)-1)]

        else:
            weights = model_agg.aggregation_update_exp(discount, learning_rate, model, H, k, num_repeat_est)

    with open('weights/weights_expw_' + str(n) + '.pkl', 'wb') as output:
        pickle.dump(weights_dict, output)
     

100%|█████████████████████████████████████████| 501/501 [03:13<00:00,  2.59it/s]

CPU times: user 32min 47s, sys: 3min 53s, total: 36min 41s
Wall time: 34min
